In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
from tqdm import tqdm

import os
import torch
from scipy.linalg import LinAlgError
import numpy as np
from torch_geometric.data import Batch, HeteroData
from scipy.sparse import coo_array

from sklearn.datasets import make_sparse_spd_matrix

In [ ]:
rng = np.random.RandomState(1)

In [ ]:
root = 'datasets/qp_100_100_0.05_0.05'
os.mkdir(root)
os.mkdir(os.path.join(root, 'processed'))

### Generic

In [ ]:
density = 0.05
Pdensity = 0.05
nrows = 100
ncols = 100

def surrogate_gen():
    assert max(nrows, ncols) * density > 1

    m, n = min(nrows, ncols), max(nrows, ncols)

    # make sure rows and cols are selected at least once
    rows = np.hstack([np.arange(m), np.random.randint(0, m, (n - m,))])
    cols = np.arange(n)

    # generate the rest
    nnz = int(nrows * ncols * density)
    num_rest = nnz - n

    rows_rest = np.random.randint(0, m, (num_rest,))
    cols_rest = np.random.randint(0, n, (num_rest,))

    values = np.random.randn(nnz)

    A = coo_array((values, (np.hstack([rows, rows_rest]), np.hstack([cols, cols_rest]))), shape=(m, n)).toarray()
    if m > n:
        A = A.T

    x_feas = np.abs(np.random.randn(ncols))  # Ensure x_feas is non-negative
    b = A @ x_feas + np.abs(np.random.randn(nrows))  # Ensure feasibility

    c = np.random.rand(ncols)
    P = make_sparse_spd_matrix(n_dim=A.shape[1], alpha=1 - Pdensity / 2.,
                               smallest_coef=0.1, largest_coef=0.9, random_state=rng).astype(np.float64)
    return P, A, b, c

bounds = None

# create ineq

In [ ]:
from utils.evaluation import numpy_inactive_contraint, normalize_cons
from qpsolvers import solve_qp

In [ ]:
ips = []
graphs = []
pkg_idx = 0
success_cnt = 0

max_iter = 15000
num = 10000

pbar = tqdm(range(max_iter))
for i in pbar:
    P, A, b, c = surrogate_gen()
    P = P / np.abs(P).max()
    c = c / (np.abs(c).max() + 1.e-10)  # does not change the result
    A, b = normalize_cons(A, b)
    
    try:
        assert np.linalg.matrix_rank(A) == min(*A.shape)
        assert np.all(np.any(A, axis=1)) and np.all(np.any(A, axis=0))
        solution = solve_qp(P, c, A, b, None, None, lb=np.zeros(c.shape[0]), ub=np.ones(c.shape[0]) * np.inf, solver="cvxopt")
    except (AssertionError, LinAlgError):
        continue
    else:
        if solution is not None:
            inactive_idx = numpy_inactive_contraint(A, b, solution)
            obj = 0.5 * solution @ P @ solution + c.dot(solution)

            P = torch.from_numpy(P).to(torch.float)
            P_where = torch.where(P)
            
            A = torch.from_numpy(A).to(torch.float)
            b = torch.from_numpy(b).to(torch.float)
            c = torch.from_numpy(c).to(torch.float)
            x = torch.from_numpy(solution).to(torch.float)

            A_where = torch.where(A)
            data = HeteroData(
                cons={
                    'num_nodes': b.shape[0],
                    'x': torch.empty(b.shape[0]),
                     },
                vals={
                    'num_nodes': c.shape[0],
                    'x': torch.empty(c.shape[0]),
                },
                cons__to__vals={'edge_index': torch.vstack(A_where),
                                'edge_attr': A[A_where][:, None]},
                vals__to__vals={'edge_index': torch.vstack(P_where),
                                'edge_attr': P[P_where][:, None]},
                x_solution=x,
                obj_solution=obj,
                q=c,
                b=b,
                inactive_idx=torch.from_numpy(inactive_idx).long(),
            )
            success_cnt += 1
            graphs.append(data)

    if len(graphs) >= 1000 or success_cnt == num:
        torch.save(Batch.from_data_list(graphs), f'{root}/processed/batch{pkg_idx}.pt')
        pkg_idx += 1
        graphs = []

    if success_cnt >= num:
        break

    pbar.set_postfix({'suc': success_cnt, 'obj': obj})

In [ ]:
from data.dataset import LPDataset

In [ ]:
ds = LPDataset(root, 'test')

In [ ]:
data = ds[0]

In [ ]:
from transforms.qp_preserve import QPScaleCoordinate
from utils.evaluation import recover_qp_from_data

In [ ]:
P,A,c,b,*_ = recover_qp_from_data(data, np.float64)
solution = solve_qp(P, c, A, b, None, None, lb=np.zeros(c.shape[0]), ub=np.ones(c.shape[0]) * np.inf, solver="cvxopt")
0.5 * solution @ P @ solution + c.dot(solution)

In [ ]:
tf = QPScaleCoordinate(1.)

In [ ]:
for _ in range(100):
    d1 = tf(data)
    P,A,c,b,*_ = recover_qp_from_data(d1, np.float64)
    solution = solve_qp(P, c, A, b, None, None, lb=np.zeros(c.shape[0]), ub=np.ones(c.shape[0]) * np.inf, solver="cvxopt")
    obj = 0.5 * solution @ P @ solution + c.dot(solution)
    print(obj)